<a href="https://colab.research.google.com/github/darkha03/Introduction_to_Machine_Learning/blob/main/tp1_mlp_from_scratch_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 — Réseau de neurones *from scratch* sur Iris (NumPy)

**Module : Deep Learning & Application — INSA CVL**  
**Durée : 2 × 1h20**

---

## Objectifs

Vous allez implémenter un petit réseau de neurones multicouches (*MLP*) **sans framework de deep learning** (uniquement NumPy) :

- propagation avant (*forward*)
- rétropropagation (*backpropagation*)
- entraînement par descente de gradient
- évaluation (accuracy, matrice de confusion)

Jeu de données : **Iris** (150 exemples, 4 features, 3 classes)

Architecture cible :

\[
4 → 8 → 3
\]

---

## Règles

**Autorisé** : `numpy`, `matplotlib`, `sklearn` (chargement données + split + métriques)  
**Interdit** : `pytorch`, `tensorflow`, `keras`

---

## À rendre

- le notebook complété (toutes les cellules exécutables)
- figures demandées (loss, accuracy, confusion matrix)
- réponses aux questions (en Markdown)

## 0) Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

np.random.seed(0)

### Si une des librairies n'éxiste pas, exécutez la commande correspondante à la librairie manquante :
!pip install numpy

!pip install matplotlib

!pip install scikit-learn

## 1) Chargement et préparation des données

### Présentation du dataset Iris

Le dataset **Iris** est un jeu de données classique en apprentissage automatique, introduit par Ronald Fisher en 1936.

Il contient :
- **150 échantillons** (fleurs)
- **3 classes** :
  - 0 → *setosa*
  - 1 → *versicolor*
  - 2 → *virginica*
- **4 caractéristiques numériques (features)** pour chaque fleur :
  1. longueur du sépale (sepal length)
  2. largeur du sépale (sepal width)
  3. longueur du pétale (petal length)
  4. largeur du pétale (petal width)

Chaque exemple est donc un vecteur :

X = [x₁, x₂, x₃, x₄]

Notre objectif est de prédire la **classe** (0, 1 ou 2) à partir de ces 4 mesures.

---

### Travail demandé
1. Charger Iris avec `load_iris()`
2. Récupérer `X` matrice de features (150 × 4) et `y` vecteur des labels (150,)
3. Normaliser les features (par exemple standardisation ou min-max)
    - **Min-Max scaling** :
     X = (X - min) / (max - min)
    - ou **Standardisation** :
     X = (X - moyenne) / écart-type
4. Avec `train_test_split`, séparer les données en :
   - 80% entraînement
   - 20% test
   
    Utiliser `stratify=y` pour garder la même proportion de classes dans train et test.
5. Convertir `y_train` et `y_test` en **one-hot** : `Y_train`, `Y_test` de forme `(N, 3)`

> **Indices :**
- Iris a 3 classes (`0,1,2`)
- One-hot : si `y=2`, alors `[0,0,1]`, si `y=1`, alors `[0,1,0]`

---

### Vérifications attendues

Après préparation :

- `X_train.shape == (N_train, 4)`
- `Y_train.shape == (N_train, 3)`
- `X_test.shape  == (N_test, 4)`
- `Y_test.shape  == (N_test, 3)`

Avec environ :
- N_train ≈ 120
- N_test ≈ 30


In [ ]:

# TODO 1: charger Iris
iris = load_iris()
X = iris['data']
y = iris['target']

# TODO 2: normalisation (choisissez une méthode simple)
# Exemple min-max :
# X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0) + 1e-12)
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0) + 1e-12)

# TODO 3: split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


def one_hot(y, num_classes):
    # TODO: retourner une matrice (len(y), num_classes)
    Y = np.zeros((len(y), num_classes))
    for i in range(len(y)):
        Y[i][y[i]] = 1
    return Y

Y_train = one_hot(y_train, 3)
Y_test  = one_hot(y_test, 3)

print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("X_test :", X_test.shape,  "Y_test :", Y_test.shape)

X_train: (120, 4) Y_train: (120, 3)
X_test : (30, 4) Y_test : (30, 3)


In [ ]:
print(y_train)

[0 0 0 0 1 0 2 2 1 2 2 1 0 1 2 2 0 1 1 0 2 0 0 2 2 1 1 0 2 2 1 1 0 2 2 1 2
 1 2 1 1 1 0 0 1 1 2 2 1 0 2 2 0 0 1 1 0 0 1 2 0 0 1 1 2 1 2 0 0 2 1 1 0 0
 2 1 2 0 1 2 2 1 2 0 1 0 0 2 2 1 2 0 0 0 0 0 1 1 1 2 0 2 0 2 0 1 1 1 1 0 2
 2 0 1 1 2 0 2 2 2]


## 2) Implémentation d'une couche linéaire

Une couche linéaire (fully-connected) lors du **Forward pass** :

$$
Z = XW + b
$$

où :
- `X` (matrice d'entrée) : $(N, \text{in\_dim})$
- `W` (matrice de poids) : $(\text{in\_dim}, \text{out\_dim})$
- `b` (vecteur de biais) : $(1, \text{out\_dim})$
- `Z` (sortie de la couche) : $(N, \text{out\_dim})$

---

### Backward pass

On suppose que l’on reçoit le gradient :

$$
\frac{\partial L}{\partial Z}
\quad \text{de dimension } (N, \text{out\_dim})
$$

On calcule alors :

#### Gradient par rapport aux poids

$$
\frac{\partial L}{\partial W} = X^\top \frac{\partial L}{\partial Z}
$$

Dimensions :
- $X^\top$ : $(\text{in\_dim}, N)$
- $\frac{\partial L}{\partial Z}$ : $(N, \text{out\_dim})$
- Résultat : $(\text{in\_dim}, \text{out\_dim})$

---

#### Gradient par rapport au biais

On somme sur le batch :

$$
\frac{\partial L}{\partial b} =
\sum_{i=1}^{N} \frac{\partial L}{\partial Z_i}
$$

Ce qui revient en pratique à :

$$
\frac{\partial L}{\partial b}
=
\sum_{i=1}^{N}
\frac{\partial L}{\partial Z}
\quad \text{(axis=0)}
$$

---

#### Gradient par rapport à l'entrée

$$
\frac{\partial L}{\partial X}
=
\frac{\partial L}{\partial Z} W^\top
$$

Dimensions :
- $(N, \text{out\_dim}) \times (\text{out\_dim}, \text{in\_dim})$
- Résultat : $(N, \text{in\_dim})$

---

### Mise à jour des paramètres

Avec un learning rate $\alpha$ :

$$
W \leftarrow W - \alpha \frac{\partial L}{\partial W}
$$

$$
b \leftarrow b - \alpha \frac{\partial L}{\partial b}
$$

---


### Travail demandé
Compléter `forward` et `backward`.



In [ ]:

class Linear:
    def __init__(self, input_dim, output_dim):
        # initialisation simple (petite variance)
        self.W = 0.01 * np.random.randn(input_dim, output_dim)
        self.b = np.zeros((1, output_dim))
        self.X = None

    def forward(self, X):
        # TODO: stocker X pour backward
        self.X = X
        Z = X.dot(self.W) + self.b
        return Z

    def backward(self, dZ, learning_rate):
        # dZ : (N, out_dim)
        # TODO: calculer dW, db, dX
        #N = dZ.shape[0]
        dW = np.transpose(self.X).dot(dZ)
        db = np.sum(dZ, axis=0)
        dX = dZ.dot(np.transpose(self.W))

        # TODO: mise à jour SGD
        self.W = self.W - learning_rate*dW
        self.b = self.b - learning_rate*db

        return dX

## 3) Activation ReLU

$$
\mathrm{ReLU}(x) = \max(0, x)
$$

### Travail demandé
Compléter `forward` et `backward`.

> Indice : en backward, le gradient passe uniquement là où l'entrée était $> 0$.

**Rappel utile pour le backward :**

$$
\frac{d}{dx}\mathrm{ReLU}(x) =
\begin{cases}
1 & \text{si } x > 0 \\
0 & \text{sinon}
\end{cases}
$$

In [ ]:
class ReLU:
    def __init__(self):
        self.input_for_backward = None # Store the input X for proper backward computation

    def forward(self, X):
        self.input_for_backward = X # Store the input X for backward pass
        out = np.maximum(0, X)
        return out

    def backward(self, dA, learning_rate=None):
        # The gradient passes only where the input X was > 0
        return dA * (self.input_for_backward > 0)

## 4) Softmax (version stable)

Pour un batch `X` de forme $(N, C)$ :

$$
\mathrm{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j} e^{x_j}}
$$

### Version vectorisée (batch)

Pour une ligne $x \in \mathbb{R}^C$ :

$$
\mathrm{softmax}(x)_i =
\frac{e^{x_i}}{\sum_{j=1}^{C} e^{x_j}}
$$

### Travail demandé
Compléter `forward`.

> Indice stabilité : soustraire $\max(X, \text{axis}=1, \text{keepdims}=True)$ avant l'exponentielle.

### Forme numériquement stable

$$
\mathrm{softmax}(x)_i =
\frac{e^{x_i - \max(x)}}{\sum_{j=1}^{C} e^{x_j - \max(x)}}
$$

In [ ]:
class Softmax:
    def __init__(self):
        self.out = None

    def forward(self, X):
        # TODO: softmax stable
        X_shift = X - np.max(X, axis=1, keepdims=True)
        exp_X = np.exp(X_shift)
        self.out = exp_X / np.sum(exp_X, axis=1, keepdims=True) # Corrected summation axis
        return self.out

    def backward(self, dY, learning_rate=None):
        # Bonus: on n'en a pas besoin si on utilise directement le gradient CE+Softmax.
        return dY

## 5) Cross-Entropy (multi-classes) + gradient

Loss cross-entropy avec labels one-hot :

$$
L = -\frac{1}{N}\sum_{i=1}^{N}\sum_{c=1}^{C} y_{ic}\,\log(\hat{y}_{ic})
$$

### Travail demandé
1. Compléter `cross_entropy_loss`

> Important : si `y_pred` vient d'un softmax, le gradient simplifié par rapport aux **logits** (avant softmax) est :

$$
\frac{\partial L}{\partial Z} = \frac{\hat{Y} - Y}{N}
$$

In [ ]:
def cross_entropy_loss(y_pred, y_true, eps=1e-12):
    '''
    y_pred: (N, C) probabilités (softmax)
    y_true: (N, C) one-hot
    '''
    # TODO: calculer la loss moyenne
    y_pred_clipped = np.where(y_pred < eps, eps, y_pred)
    loss = -1/len(y_true) * sum(
            sum(y1_j * np.log(y2_j) for y1_j, y2_j in zip(y1, y2))
            for y1, y2 in zip(y_true, y_pred_clipped)
            )

    return loss


## 6) Construction du réseau

Architecture :

- $\mathrm{Linear}(4 \rightarrow 8)$
- $\mathrm{ReLU}$
- $\mathrm{Linear}(8 \rightarrow 3)$

### Forme compacte

$$
X \in \mathbb{R}^{N \times 4}
\;\xrightarrow{\;\mathrm{Linear}\;}
\mathbb{R}^{N \times 8}
\;\xrightarrow{\;\mathrm{ReLU}\;}
\mathbb{R}^{N \times 8}
\;\xrightarrow{\;\mathrm{Linear}\;}
\mathbb{R}^{N \times 3}
\;\xrightarrow{\;\mathrm{Softmax}\;}
\hat{Y} \in \mathbb{R}^{N \times 3}
$$

### Équations

$$
Z_1 = XW_1 + b_1
$$

$$
A_1 = \mathrm{ReLU}(Z_1)
$$

$$
Z_2 = A_1 W_2 + b_2
$$

$$
\hat{Y} = \mathrm{softmax}(Z_2)
$$

In [ ]:
network = [
    Linear(4, 8),
    ReLU(),
    Linear(8, 3),
    Softmax()
]

## 7) Forward / Backward génériques

- `forward` applique chaque couche dans l'ordre
- `backward` applique les couches en sens inverse

In [ ]:
def forward(network, X):
    out = X
    for layer in network:
        out = layer.forward(out)
    return out

def backward(network, grad, learning_rate):
    for layer in reversed(network):
        grad = layer.backward(grad, learning_rate)

## 8) Entraînement

### Travail demandé
Implémenter la boucle d'entraînement :

1. Forward sur `X_train`
2. Calculer la loss cross-entropy
3. Calculer le gradient simplifié CE + Softmax
4. Backward + mise à jour des poids
5. Stocker loss et accuracy

Hyperparamètres suggérés :
- `epochs = 500`
- `learning_rate = 0.5`

### Rappels (pour clarifier les formes)

Si le réseau produit des logits $Z \in \mathbb{R}^{N \times C}$ (avant softmax) et des prédictions
$\hat{Y} = \mathrm{softmax}(Z)$, alors la cross-entropy multi-classes (one-hot) est :

$$
L = -\frac{1}{N}\sum_{i=1}^{N}\sum_{c=1}^{C} y_{ic}\,\log(\hat{y}_{ic})
$$

Le gradient simplifié (CE + Softmax) **par rapport aux logits** $Z$ est :

$$
\frac{\partial L}{\partial Z} = \frac{\hat{Y} - Y}{N}
$$

> **Note technique** : le gradient simplifié CE+Softmax est calculé wrt les logits **avant** softmax.  
> Comme notre réseau termine par `Softmax()`, on appliquera le backward sur `network[:-1]` (on “saute” la couche Softmax en backward).

In [ ]:
def predict_proba(network, X):
    return forward(network, X)

def predict_label(network, X):
    proba = predict_proba(network, X)
    return np.argmax(proba, axis=1)

epochs = 500
learning_rate = 0.5

loss_history = []
acc_history = []

for epoch in range(1, epochs + 1):
    # TODO: forward
    y_pred = predict_proba(network, X_train)
    # TODO: loss
    loss = cross_entropy_loss(y_pred, Y_train)
    loss_history.append(loss)

    # TODO: accuracy train
    y_hat = predict_label(network, X_train)
    acc = sum(y1 == y2 for y1, y2 in zip(y_hat, y_train)) / len(y_hat)
    acc_history.append(acc)

    # TODO: gradient CE+Softmax (wrt logits)
    grad = (y_pred - Y_train)/len(y_pred)
    # TODO: backward (sans la couche Softmax)
    backward(network[:-1], grad, learning_rate)

    if epoch % 25 == 0:
        print(f"Epoch {epoch:03d} | loss={loss:.4f} | acc={acc:.3f}")

Epoch 025 | loss=1.0897 | acc=0.667
Epoch 050 | loss=0.6864 | acc=0.700
Epoch 075 | loss=0.4247 | acc=0.883
Epoch 100 | loss=0.3098 | acc=0.942
Epoch 125 | loss=0.2310 | acc=0.950
Epoch 150 | loss=0.1801 | acc=0.950
Epoch 175 | loss=0.1481 | acc=0.950
Epoch 200 | loss=0.1371 | acc=0.958
Epoch 225 | loss=0.1368 | acc=0.958
Epoch 250 | loss=0.1021 | acc=0.967
Epoch 275 | loss=0.0936 | acc=0.975
Epoch 300 | loss=0.0872 | acc=0.975
Epoch 325 | loss=0.0821 | acc=0.975
Epoch 350 | loss=0.0780 | acc=0.975
Epoch 375 | loss=0.0747 | acc=0.975
Epoch 400 | loss=0.0720 | acc=0.975
Epoch 425 | loss=0.0697 | acc=0.975
Epoch 450 | loss=0.0678 | acc=0.975
Epoch 475 | loss=0.0661 | acc=0.975
Epoch 500 | loss=0.0647 | acc=0.975


In [ ]:
# TODO: prédictions sur test
y_test_pred = predict_label(network, X_test)
test_acc = sum(y1 == y2 for y1, y2 in zip(y_test_pred, y_test)) / len(y_test_pred)
print("Accuracy test:", test_acc)

Accuracy test: 1.0


## 11) Questions (à compléter)

Répondez ici :

1. Quel est le rôle de ReLU ?
2. Pourquoi utilise-t-on Softmax en sortie ?
3. Quel est le rôle de la cross-entropy ?
4. Que se passe-t-il si le learning rate est trop grand ? Trop petit ?
5. Pourquoi séparer train/test ?

## 11) Questions (à compléter)

Répondez ici :

1.  **Quel est le rôle de ReLU ?**
    La fonction d'activation ReLU (Rectified Linear Unit) introduit de la non-linéarité dans le réseau de neurones. Sans elle, le réseau se comporterait comme un simple modèle linéaire, quelles que soient le nombre de couches. ReLU permet au réseau d'apprendre des relations complexes dans les données en activant seulement les neurones dont l'entrée est positive, ce qui aide également à résoudre le problème de la disparition du gradient (vanishing gradient problem) par rapport à d'autres fonctions comme la sigmoïde ou la tanh pour les valeurs positives.

2.  **Pourquoi utilise-t-on Softmax en sortie ?**
    Softmax est utilisée en sortie pour les problèmes de classification multi-classes. Elle convertit un vecteur de nombres réels (logits) en une distribution de probabilité. Cela signifie que la somme des sorties sera égale à 1, et chaque sortie peut être interprétée comme la probabilité que l'entrée appartienne à une classe spécifique. C'est particulièrement utile pour obtenir des prédictions claires et interprétables pour chaque classe.

3.  **Quel est le rôle de la cross-entropy ?**
    La cross-entropy est une fonction de perte (loss function) couramment utilisée pour les problèmes de classification. Elle mesure la différence entre la distribution de probabilité prédite par le modèle (softmax en sortie) et la distribution de probabilité réelle (labels one-hot). L'objectif de l'entraînement est de minimiser cette perte, ce qui pousse le modèle à faire des prédictions plus proches des vrais labels. Une valeur de cross-entropy faible indique que le modèle est confiant et correct dans ses prédictions.

4.  **Que se passe-t-il si le learning rate est trop grand ? Trop petit ?**
    *   **Trop grand :** Si le learning rate est trop grand, le modèle risque de

    "sauter" par-dessus le minimum de la fonction de perte, ce qui peut entraîner une divergence de l'entraînement ou un comportement très instable avec la perte qui oscille fortement.
    *   **Trop petit :** Si le learning rate est trop petit, le modèle convergera très lentement. L'entraînement prendra beaucoup plus de temps pour atteindre un bon niveau de performance, et il pourrait même rester bloqué dans un minimum local sans jamais atteindre la solution optimale.

5.  **Pourquoi séparer train/test ?**
    Séparer les données en ensembles d'entraînement (train) et de test (test) est crucial pour évaluer la capacité de généralisation d'un modèle. L'ensemble d'entraînement est utilisé pour ajuster les poids du modèle. L'ensemble de test, quant à lui, est composé de données que le modèle n'a jamais vues pendant l'entraînement. En évaluant le modèle sur cet ensemble, on peut obtenir une estimation impartiale de ses performances sur de nouvelles données et détecter le surapprentissage (overfitting), où le modèle a mémorisé les données d'entraînement au lieu d'apprendre les motifs sous-jacents.

## Bonus (optionnel)

1. Tester d'autres tailles de couche cachée : 4→4→3, 4→16→3
2. Ajouter une deuxième couche cachée : 4→8→8→3
3. (Optionnel) Ajouter une régularisation L2 sur les poids